In [ ]:
pip install fake-useragent

In [ ]:
pip install SPARQLWrapper

In [3]:
from fake_useragent import UserAgent
from os.path import join,isfile
from os import listdir
import dask
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
from collections import Counter
import json
import re
import time
import bookquery
import sys
import os
import datetime
import math

ua = UserAgent()#Прокси
curr_directory = os.getcwd()
new_path = join(curr_directory,"Information")
try:# Создаем базовую директорию для файлов
    os.mkdir(new_path)
except OSError:
    print ("Директория %s уже создана" % new_path)
    
def execute_query(qquery, entity_id):
    
    """Работа с запросами"""
    
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql",agent = ua.random)
    sparql.setQuery(qquery.format(entity_id = entity_id))
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results['results']['bindings']

def counter_for_files(path):
    
    """Подсчет файлов в директории"""
    
    
    onlyfiles = [join(path, f) for f in listdir(path) if isfile(join(path, f))]
    return onlyfiles

Директория C:\Users\Алексей Кушнер\Sberbank\Geography\Big\Information уже создана


In [11]:
def counter_popularity(v , f = 0,entity_id = None,label  = None):
    
    """Функция возвращает популярность для каждой сущности"""
    
    if f > 10:#Если не удается нормально проработать , то записываем -1
        new_entity = {
            'entity':entity_id,
            'label':label,
            'popularity' : -1
        }
        return new_entity
    try:
        new_entity = {}
        entity_id, label = v['entity'], v['label']
        from_res =int(execute_query(bookquery.counter_from, entity_id)[0]['countOut']['value'])#Исходящие ссылки
        to_res =int(execute_query(bookquery.counter_to, entity_id)[0]['countIn']['value']) #Входящие ссылки
        new_entity = {
            'entity':entity_id,
            'label':label,
            'popularity' : from_res + to_res#общая популярность
        }
        return  new_entity
    except Exception as e:#перехватываем ошибку и слипим на 3 секунды для более корректной работы endpoint
        time.sleep(3)
        return counter_popularity(v, f + 1,entity_id, label)
        
    
        

def creating_parts(i, ids, file_name,path):
    
    """Создает блоки по 50000 представителей с проставленной популярностью"""
    
    new_path = join(curr_directory,join("Information",path +"\\data\\data_with_popularity"))
    if str(i) not in list(map (lambda x : x.split('_')[-1].split('.')[0],counter_for_files(new_path))):
        print(f'Файл {file_name}_with_popularity_{i} начал обрабатываться {datetime.datetime.now().time()}')
        start = time.time()
        res = []
        result = []   
        for k, v in ids.iterrows():#Запросы по каждой сущности
            res.append(dask.delayed(counter_popularity)(v))
    #           result.append(counter_popularity(v))
        if not res:
            print(f"Файл {label} является пустым")
            return None
        result = pd.DataFrame(data = dask.compute(*res))
    #       result = pd.DataFrame(data = result)
        result.sort_values(by = 'popularity',ascending = False).to_csv(
                join(new_path,file_name + f'_with_popularity_{i}.csv'),
                encoding='utf-8-sig',index = False,sep = ';')# Формирование
        end = time.time()
        print(f'Формирование файла {file_name}_with_popularity_{i} окончено , заняло {(end-start)/60} минут\n')
    else:
        print(f'{i} файл уже обработан')

def determ_popul(filename,path):
    
    """Функция для нахождения популярности в каждом блоке"""
    
    BLOCK_SIZE = 50000 #Размер блока
    curr_directory = os.getcwd()
    new_path = join(curr_directory,join("Information",path + "\\data\data_with_popularity"))
    labels = filename.split('\\')[-1].split('_')#название класса
    label = labels[-2]
    entity_id = labels[-1].split('.')[0]
    if filename not in counter_for_files(new_path):# проверка на наличие     
        start = time.time()
        file_name = filename.split('\\')[-1].split('.')[0]
        print(f'Файл {label} начал обрабатываться {datetime.datetime.now().time()}')
        ids = pd.read_csv(filename, encoding='utf8',sep = ';')#читаем файл
        
        for i in range(math.ceil(len(ids)/BLOCK_SIZE)): # Обход всех элементов блока
            if (i+1) *BLOCK_SIZE< len(ids):
                creating_parts(i , ids[BLOCK_SIZE * i: BLOCK_SIZE*(i+1)] ,file_name,path)
            else:
                creating_parts(i , ids[BLOCK_SIZE * i:],file_name,path)
        end = time.time()
        
        print(f'Формирование файла {file_name}_with_popularity окончено , заняло {(end-start)/60} минут\n!!!!!!!!!!!!!!!!!')
    else:
        print(f'Файл {label} уже обработан')

def global_finding_popularity(path):
    
    """Обработка файла в директории"""
    
    res = []
    curr_directory = os.getcwd()
    new_path = join(curr_directory,join("Information",path + "\\data\data_with_popularity"))
    path_for_finding = join(curr_directory,join("Information",path + "\\data"))
    try:
        os.mkdir(new_path)
    except OSError:
        print ("Директория %s уже создана" % new_path)
    for filename in counter_for_files(path_for_finding):
        determ_popul(filename,path)
    #result = dask.compute(*res)

global_finding_popularity("2")

Директория C:\Users\Алексей Кушнер\Sberbank\Geography\Big\Information\2\data\data_with_popularity уже создана
Файл country начал обрабатываться 12:00:15.951563
Файл entities_country_Q6256_with_popularity_0 начал обрабатываться 12:00:15.954562
EndPointInternalError: endpoint returned code 500 and response. 

Response:
b'SPARQL-QUERY: queryStr=\nSELECT  (COUNT(?obj) AS ?countIn) \n\t\t\tWHERE {\n\t\t\t  VALUES (?book) {(wd:Q30)} .\n\t\t\t  ?obj ?prop_id ?book.\n\t\t\t  ?wd wikibase:directClaim ?prop_id.\n\t\t\t  ?wd rdfs:label ?prop_label.\n\t\t\t  FILTER((LANG(?prop_label)) = "ru").\n\t\t\t}\n\njava.util.concurrent.TimeoutException\n\tat java.util.concurrent.FutureTask.get(FutureTask.java:205)\n\tat com.bigdata.rdf.sail.webapp.BigdataServlet.submitApiTask(BigdataServlet.java:292)\n\tat com.bigdata.rdf.sail.webapp.QueryServlet.doSparqlQuery(QueryServlet.java:678)\n\tat com.bigdata.rdf.sail.webapp.QueryServlet.doGet(QueryServlet.java:290)\n\tat com.bigdata.rdf.sail.webapp.RESTServlet.doGe

Формирование файла entities_country_Q6256_with_popularity_0 окончено , заняло 4.661867006619771 минут

Файл entities_country_Q6256_with_popularity_1 начал обрабатываться 12:04:55.666582
Формирование файла entities_country_Q6256_with_popularity_1 окончено , заняло 6.658766269683838 минут

Файл entities_country_Q6256_with_popularity_2 начал обрабатываться 12:11:35.192558
Формирование файла entities_country_Q6256_with_popularity_2 окончено , заняло 17.094158029556276 минут

Файл entities_country_Q6256_with_popularity_3 начал обрабатываться 12:28:40.842040
Формирование файла entities_country_Q6256_with_popularity_3 окончено , заняло 11.723495014508565 минут

Файл entities_country_Q6256_with_popularity_4 начал обрабатываться 12:40:24.251741


KeyboardInterrupt: 

In [3]:
def execute_query(qquery, book_id):
    
    """Работа с запросами"""
    
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql",agent = ua.random)
    sparql.setQuery(qquery.format(book_id = book_id))
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results['results']['bindings']

def entity_from_sparql(results):
    
    """Обработка исходящих предикатов"""
    
    entity = {}
    for result in results:#Построчно читаем
        wdLabel, ps_Label, ps_, wd, direction = result['wdLabel']['value'], result['ps_Label']['value'], \
                        result['ps_']['value'], result['wd']['value'], result['direction']['value']
        ps = {
            'propValueID': ps_,
            'propValue': ps_Label
        }
        if wdLabel in entity:# Формируем словари для повторяющихся предикатов
                entity[wdLabel]['content'].append(ps)
        else:
            entity[wdLabel] = {
                'propID': wd,
                'direction': direction,
                'content': [ps]
            }
    return entity

def entity_to_sparql(results):
    
    """Обрабокта входящих предикатов"""
    
    entity = {}
    for result in results:
        obj, objLabel, wd, prop_label, bookLabel, direction = result['obj']['value'], result['objLabel']['value'], \
                        result['wd']['value'], result['prop_label']['value'], \
                        result['bookLabel']['value'], result['direction']['value']
        ps = {
            'propValueID': obj,
            'propValue': objLabel
        }
        
        if prop_label in entity:
                entity[prop_label]['content'].append(ps)
        else:
            entity[prop_label] = {
                'propID': wd,
                'direction': direction,
                'content': [ps]
            }
    return entity
    

def get_properties_of_book(book_id, book_name=None):
    
    """Формирование датафрейма и JSON """
    
    results_from = execute_query(bookquery.query_from, book_id)
    results_to = execute_query(bookquery.query_to, book_id)
    
    entity_from = entity_from_sparql(results_from)
    entity_to = entity_to_sparql(results_to)
    
    entity = {**entity_from, **entity_to}# Объединяем словарь 

    _book_name = book_name
    
    counter, res = {}, []
    for k, v in entity.items(): 
        counter[k] = len(v['content'])
    for k, v in counter.items():# Формирвоание датафрейма
        prop = {
            'entity_id': book_id,
            'entity_name': _book_name,
            'prop_name': k,
            'prop_count':1,
            'prop_count_value': v,
            'prop_dir': entity[k]['direction']
        }
        res.append(prop)
    return pd.DataFrame(res)

def processing(v, f=0):# 
    
    """Обработка запроса"""
    
    if f > 20:# Если не удается , то ничего не возвращаем
            return None
    try:
        book_id, bookLabel = v['entity'], v['label']
        dataframe= get_properties_of_book(book_id, book_name=bookLabel)
      
        return  dataframe
    except Exception as e:
        time.sleep(3)
        return processing(v, f + 1)


        
def entity_analyzer(filename,path):
    
    """Создание csv-файлов"""
    
    curr_path = join(curr_directory,join("Information",path + "\\analyzing")) # Обработчик файлов
    Label = filename.split('\\')[-1].split('.')[0].split('_')[-1]# Название файла
    file_name = filename.split('\\')[-1].split('.')[0].split('_')[0:3]
    file_name = '_'.join(file_name)
    label  = filename.split('\\')[-1].split('.')[0].split('_')[-2]
    if Label not in list (map(lambda x:x.split('\\')[-1].split('.')[0].split('_')[-1],counter_for_files(curr_path))):
        print(f'Файл номер {Label} начал обрабатываться {datetime.datetime.now().time()}')
        start = time.time()
        ids = pd.read_csv(filename, encoding='utf8',sep = ';')
        stats_dfs = []
        entities = []
        res = []
        result = []
        for k, v in ids.iterrows():#Запросы по каждой сущности
#             result.append(processing(v))
            res.append(dask.delayed(processing)(v))
        if res == []:
            print(f'Файл {Label} пуст')
            return None
        result = dask.compute(*res)
        errors_counter = 0
        for i in range(len(result)):#Формируем датафреймы
            try:
                stats_dfs.append(result[i])
            except Exception as e:
                errors_counter+=1
                continue
        #Создание файла со всеми предикатами для каждой сущности        
       
        stats_df = pd.concat(stats_dfs)#Создание файлов
        stats_df.sort_values(by = 'prop_count',ascending = False).to_csv(
            curr_path + f'/analyzed_{file_name}_{Label}.csv', encoding='utf-8-sig', index=False,sep = ';')
# Создание агрегированного по сущностям файла для предикатов
        stats_aggregated = stats_df.groupby('prop_name').sum().sort_values(by = ['prop_count','prop_count_value'],ascending = False)
        stats_aggregated['prop_counter'] = stats_aggregated['prop_count']/(len(ids)-errors_counter) * 100
        stats_aggregated.to_csv(curr_path + f'/aggregated/{file_name}_aggregated_{Label}.csv', encoding='utf-8-sig',sep = ';')



        end = time.time()
        print(f'Файл {file_name}_{Label} отработал:количество необработанных файлов {errors_counter} из {len(ids)}(успех:{(len(ids)-errors_counter)*100/(len(ids))} %), заняло {(end-start)/60} минут')
    else:
        print(f'Файл {Label} уже обработан')


        
def global_analyzer(path):
    
    """Главная функция обработки файлов"""
    
    res = []
    curr_directory = os.getcwd()
    
    new_path_main = join(curr_directory,join("Information",path + r"\analyzing"))
    path_for_finding = join(curr_directory,join("Information",path + r"\data\data_with_popularity"))
    try:
        os.mkdir(new_path_main)
    except OSError:
        print ("Директория %s уже создана" % new_path_main)

    new_path = join(new_path_main,'aggregated')
    try:
        os.mkdir(new_path)
    except OSError:
        print ("Директория %s уже создана" % new_path)

    new_path = join(new_path_main,'aggregated_with_popularity')
    try:
        os.mkdir(new_path)
    except OSError:
        print ("Директория %s уже создана" % new_path)
    for filename in counter_for_files(path_for_finding):
        entity_analyzer(filename,path)
        #res.append(dask.delayed( entity_analyzer)(filename,path))
    #result = dask.compute(*res)


global_analyzer("2")


Директория C:\Users\Алексей Кушнер\Sberbank\Geography\Big\Information\2\analyzing уже создана
Директория C:\Users\Алексей Кушнер\Sberbank\Geography\Big\Information\2\analyzing\aggregated уже создана
Директория C:\Users\Алексей Кушнер\Sberbank\Geography\Big\Information\2\analyzing\aggregated_with_popularity уже создана
Файл 0 уже обработан
Файл 1 уже обработан
Файл 2 уже обработан
